<a href="https://colab.research.google.com/github/rafabrnd/Coursera_Capstone/blob/master/coursera_capstone_Kmeans_torrontoneighborhoods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Segmenting and Clustering Neighborhoods in Toronto**
### *Applied Data Science Capstone > Week 3*






<img src="https://media.giphy.com/media/olAik8MhYOB9K/giphy.gif" width="250" align="left">



Load the necessary packages

In [0]:
import numpy as np
import pandas as pd


### 1. Load the file URL from GitHub
Ignore some bad lines where there was some formating errors. To get to this dataset I copied the table into MS Excel and saved the file as .csv (comma separated)  

In [67]:
url = 'https://raw.githubusercontent.com/rafabrnd/Coursera_Capstone/master/toronto_neigh_dataset.csv' 
toronto = pd.read_csv(url,sep=',',error_bad_lines=False) #here i ignored some bad lines to make it possible to load the file
toronto.head() #check the first 5 rows of the dataset

b'Skipping line 15: expected 3 fields, saw 4\n'


,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [68]:
#check the shape of the dataset

print("Dataset shape is: ", toronto.shape)
print ('Columns names are: ', toronto.columns)


Dataset shape is:  (182, 3)
Columns names are:  Index(['Postal code ', 'Borough ', 'Neighborhood'], dtype='object')


### 2. Data wrangling

Here I noticed that the 'Postal code ' and 'Borough ' have a space after their name. 
Their names should be replaced for something easier to read


In [69]:
toronto = toronto.rename(columns={'Postal code ':'postal_code','Borough ':'borough', 'Neighborhood':'neighborhood'})
toronto.columns

Index(['postal_code', 'borough', 'neighborhood'], dtype='object')

Here all the entries where the borough is 'Not assinged ' are droped.
Notice that there is a ' ' at the right side.
A new dataframe is defined with all the rows are that have assigned borough names

In [0]:
toronto = toronto.loc[~(toronto['borough'] == 'Not assigned ')]

In [74]:
print("The current shape of the Toronto datafrane is: ", toronto.shape)
toronto.head()

The current shape of the Toronto datafrane is:  (105, 3)


,postal_code,borough,neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


Due to the copy & paste from the wikipedia page into the MS Excel there were some residual spaces either before or after some of the values. Here I used a **for** loop that loops through the different column names and strips each Series from potential spaces (' ') that could be located at their left or right side.
Note that this strip() function does not remove the spaces in the middle of the strings, so it is safe to use even though we have some 2 work names. 

In [73]:
columns = list(toronto.columns)
for column_name in columns:
  toronto[column_name] = toronto[column_name].str.strip()

#this following piece of code defines a new df where all the ' ' are replaced with '_'
#it does not change the 'toronto' dataframe
#this was done to test the for loop above
test_df = toronto.replace(' ','_', regex=True)
test_df

The current shape of the Toronto datafrane is:  (105, 3)


Here I split the neighborhood names at the '/' in order to create a list with each one of those names

Make a loop where each one of those names is assigned to the same 'postal_code' and 'borough' they were originally at. 

In [77]:
toronto_expanded = toronto['neighborhood'].str.split('/', n=-1)
toronto_expanded.head()

2                                        [Parkwoods]
3                                 [Victoria Village]
4                      [Regent Park ,  Harbourfront]
5               [Lawrence Manor ,  Lawrence Heights]
6    [Queen's Park ,  Ontario Provincial Government]
Name: neighborhood, dtype: object

In [84]:
toronto_expanded = toronto_expanded.reset_index(drop=True)
toronto_expanded.head()

0                                        [Parkwoods]
1                                 [Victoria Village]
2                      [Regent Park ,  Harbourfront]
3               [Lawrence Manor ,  Lawrence Heights]
4    [Queen's Park ,  Ontario Provincial Government]
Name: neighborhood, dtype: object

In [89]:
#each value in the series is now a Nested List and (as an example) we can access their values using a nested list index:
toronto_expanded.loc[2][1] #

' Harbourfront'

In [0]:
#remove the spaces before and after each of the values using str.strip
